# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [89]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from config import weather_api_key, gkey
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [102]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

631

In [103]:
cities_list = []
for city in cities: 
    cities_list.append(city.title())
    
print(cities_list)

['Punta Arenas', 'Barentsburg', 'Esperance', 'Yellowknife', 'Busselton', 'Hasaki', 'Cherskiy', 'Aksarka', 'Georgetown', 'Phan Rang', 'Tasiilaq', 'Hermanus', 'Sedalia', 'Kahului', 'Bokoro', 'Ushuaia', 'Nizhneyansk', 'Grindavik', 'New Norfolk', 'Acapulco', 'Cabo San Lucas', 'Puerto Ayora', 'Hilo', 'Ribeira Grande', 'Mocuba', 'Yuzhno-Yeniseyskiy', 'Makakilo City', 'Barbar', 'Klaksvik', 'Torbay', 'Pitsunda', 'Mataura', 'Inhambane', 'Chuy', 'Olkhovka', 'Richards Bay', 'Kapaa', 'Upernavik', 'Cap Malheureux', 'Sovetskiy', 'Souillac', 'Nikolskoye', 'Cape Town', 'Colares', 'Palabuhanratu', 'Tezu', 'Bridgetown', 'Atar', 'Thompson', 'Arraial Do Cabo', 'Grand Baie', 'Tilichiki', 'Rikitea', 'Ust-Tsilma', 'Bredasdorp', 'Urucui', 'Artyk', 'Vaini', 'Dunedin', 'Paamiut', 'La Palma', 'Nizhniy Kuranakh', 'Atuona', 'Yilan', 'Dargaville', 'Kaitangata', 'Pevek', 'Parrita', 'Dharchula', 'Taolanaro', 'Kushima', 'Zhigansk', 'Amantea', 'Fairbanks', 'Sorvag', 'Dikson', 'Carnarvon', 'Amderma', 'Caravelas', 'Gravd

In [116]:
# city_test = ["Denver", "Pittsburgh"]
# test_df = pd.DataFrame(city_test)
# test_df

cities_df = pd.DataFrame(cities_list)
cities_df = cities_df.rename(columns={0: "City"})
cities_df

,City
0,Punta Arenas
1,Barentsburg
2,Esperance
3,Yellowknife
4,Busselton
...,...
626,San Alberto
627,Karasburg
628,Lavrentiya
629,Nyurba


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [48]:
# Get: Temperature, Humidity, Cloudiness, Windspeed
base_weather_url = "http://api.openweathermap.org/data/2.5/weather?q="

weather_data_dict = {}

In [63]:
for city in city_test:
    time.sleep(1)
    get_weather_url = base_weather_url + f"{city}&appid={weather_api_key}&units=imperial"
    weather_request = requests.get(get_weather_url).json()
    temp = weather_request["main"]["temp"]
    humid = weather_request["main"]["humidity"]
    weather_data_dict.update({f"{city}": {"Temperature": temp, "Humidity": humid}})
    print(get_weather_url)
    pprint(weather_request)

http://api.openweathermap.org/data/2.5/weather?q=Denver&appid=2d553e63a551d7ef27c92c3e368247d9&units=imperial
{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 39.74, 'lon': -104.98},
 'dt': 1603140891,
 'id': 5419384,
 'main': {'feels_like': 61.38,
          'humidity': 21,
          'pressure': 1015,
          'temp': 69.42,
          'temp_max': 71.6,
          'temp_min': 68},
 'name': 'Denver',
 'sys': {'country': 'US',
         'id': 3449,
         'sunrise': 1603113285,
         'sunset': 1603152868,
         'type': 1},
 'timezone': -21600,
 'visibility': 10000,
 'weather': [{'description': 'broken clouds',
              'icon': '04d',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 50, 'speed': 6.93}}
http://api.openweathermap.org/data/2.5/weather?q=Pittsburgh&appid=2d553e63a551d7ef27c92c3e368247d9&units=imperial
{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 40.44, 'lon': -80},
 'dt': 160314105

In [66]:
weather_df = pd.DataFrame(weather_data_dict)
weather_df

,Denver,Pittsburgh
Temperature,69.42,54.61
Humidity,21.00,100.00


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression